In [ ]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Reshape, Bidirectional, LSTM, Dense, Lambda, Activation, \
    BatchNormalization, Dropout
from keras.optimizers import Adam


In [ ]:
# load dataset

train_size = 30000
valid_size = 3000

train_x = np.load('/content/drive/MyDrive/text_recognition/train_x.npy')
valid_x = np.load('/content/drive/MyDrive/text_recognition/valid_x.npy')


train_y = np.load('/content/drive/MyDrive/text_recognition/train_y.npy')
train_label_len = np.load('/content/drive/MyDrive/text_recognition/train_label_len.npy')
train_input_len = np.load('/content/drive/MyDrive/text_recognition/train_input_len.npy')
train_output = np.load('/content/drive/MyDrive/text_recognition/train_output.npy')


valid_y = np.load('/content/drive/MyDrive/text_recognition/valid_y.npy')
valid_label_len = np.load('/content/drive/MyDrive/text_recognition/valid_label_len.npy')
valid_input_len = np.load('/content/drive/MyDrive/text_recognition/valid_input_len.npy')
valid_output = np.load('/content/drive/MyDrive/text_recognition/valid_output.npy')

In [ ]:
def preprocess(img):
    (h, w) = img.shape

    final_img = np.ones([64, 256]) * 255  # blank white image

    # crop
    if w > 256:
        img = img[:, :256]

    if h > 64:
        img = img[:64, :]

    final_img[:h, :w] = img
    return cv2.rotate(final_img, cv2.ROTATE_90_CLOCKWISE)


In [ ]:
alphabets = u"ABCDEFGHIJKLMNOPQRSTUVWXYZ-' "
max_str_len = 24  # max length of input labels
num_of_characters = len(alphabets) + 1  # +1 for ctc pseudo blank
num_of_timestamps = 64  # max length of predicted labels


def label_to_num(label):
    label_num = []
    for ch in label:
        label_num.append(alphabets.find(ch))

    return np.array(label_num)


def num_to_label(num):
    ret = ""
    for ch in num:
        if ch == -1:  # CTC Blank
            break
        else:
            ret += alphabets[ch]
    return ret

In [ ]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [ ]:
# model itself
input_data = Input(shape=(256, 64, 1), name='input')

inner = Conv2D(32, (3, 3), padding='same', name='conv1', kernel_initializer='he_normal')(input_data)
inner = BatchNormalization()(inner)
inner = Activation('relu')(inner)
inner = MaxPooling2D(pool_size=(2, 2), name='max1')(inner)

inner = Conv2D(64, (3, 3), padding='same', name='conv2', kernel_initializer='he_normal')(inner)
inner = BatchNormalization()(inner)
inner = Activation('relu')(inner)
inner = MaxPooling2D(pool_size=(2, 2), name='max2')(inner)
inner = Dropout(0.3)(inner)

inner = Conv2D(128, (3, 3), padding='same', name='conv3', kernel_initializer='he_normal')(inner)
inner = BatchNormalization()(inner)
inner = Activation('relu')(inner)
inner = MaxPooling2D(pool_size=(1, 2), name='max3')(inner)
inner = Dropout(0.3)(inner)

# CNN to RNN
inner = Reshape(target_shape=((64, 1024)), name='reshape')(inner)
inner = Dense(64, activation='relu', kernel_initializer='he_normal', name='dense1')(inner)

## RNN
inner = Bidirectional(LSTM(256, return_sequences=True), name='lstm1')(inner)
inner = Bidirectional(LSTM(256, return_sequences=True), name='lstm2')(inner)

## OUTPUT
inner = Dense(num_of_characters, kernel_initializer='he_normal', name='dense2')(inner)
y_pred = Activation('softmax', name='softmax')(inner)

model = Model(inputs=input_data, outputs=y_pred)

labels = Input(name='gtruth_labels', shape=[max_str_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')

ctc_loss = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])
model_final = Model(inputs=[input_data, labels, input_length, label_length], outputs=ctc_loss)

model_final.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=Adam(lr=0.0001))
model_final.fit(x=[train_x, train_y, train_input_len, train_label_len], y=train_output,
                validation_data=([valid_x, valid_y, valid_input_len, valid_label_len], valid_output), epochs=60, batch_size=128)

model.save('/content/drive/MyDrive/text_recognition/right_model.h5')

Epoch 1/60
235/235 [==============================] - 46s 195ms/step - loss: 24.6048 - val_loss: 20.6288
Epoch 2/60
235/235 [==============================] - 44s 189ms/step - loss: 20.1959 - val_loss: 20.0952
Epoch 3/60
235/235 [==============================] - 45s 189ms/step - loss: 19.8385 - val_loss: 19.6994
Epoch 4/60
235/235 [==============================] - 44s 189ms/step - loss: 19.3918 - val_loss: 19.2739
Epoch 5/60
235/235 [==============================] - 45s 190ms/step - loss: 18.6316 - val_loss: 18.8819
Epoch 6/60
235/235 [==============================] - 44s 188ms/step - loss: 17.7229 - val_loss: 17.8359
Epoch 7/60
235/235 [==============================] - 44s 189ms/step - loss: 16.5887 - val_loss: 16.1010
Epoch 8/60
235/235 [==============================] - 45s 190ms/step - loss: 14.8906 - val_loss: 14.1356
Epoch 9/60
235/235 [==============================] - 45s 190ms/step - loss: 12.8602 - val_loss: 11.5135
Epoch 10/60
235/235 [==============================] - 